# Lab Workbook No 7: Spatial Microsimulation - Simple World


In [1]:
import pandas as pd
import numpy as np


num_individuals = 1000


ages = np.random.randint(2, 90, size=num_individuals)
sex = np.random.choice(['m', 'f'], size=num_individuals)
income = np.random.randint(1000, 10000, size=num_individuals)


synthetic_data = pd.DataFrame({'age': ages, 'sex': sex, 'income': income})
synthetic_data.head()

,age,sex,income
0,63,m,6325
1,50,f,5353
2,66,f,3462
3,45,f,7088
4,85,m,4236


In [2]:
import pandas as pd
import numpy as np
import copy

In [3]:
ind = pd.read_csv('/UA/Lab 7 Data /SimpleWorld/ind-full.csv')
ind

,id,age,sex,income
0,1,59,m,2868
1,2,54,m,2474
2,3,35,m,2231
3,4,73,f,3152
4,5,49,f,2473


In [4]:
con_sex = pd.read_csv('/UA/Lab 7 Data /SimpleWorld/sex.csv')
con_age = pd.read_csv('/UA/Lab 7 Data /SimpleWorld/age.csv')

In [5]:
con_sex

,m,f
0,6,6
1,4,6
2,3,8


In [6]:
con_age

,a0.49,a.50+
0,8,4
1,2,8
2,7,4


In [7]:

print ("con_age total:", con_age.values.sum(), 
       "; con_sex total:", con_sex.values.sum())

print ("con_age row sums:", con_age.sum(axis=1).values, 
       "; con_sex row sums:", con_sex.sum(axis=1).values)


con_age.sum(axis=1) == con_sex.sum(axis=1)

con_age total: 33 ; con_sex total: 33
con_age row sums: [12 10 11] ; con_sex row sums: [12 10 11]


0    True
1    True
2    True
dtype: bool

In [8]:
ind_orig = ind.copy()

ind = ind.drop(['income'],axis=1)
ind

,id,age,sex
0,1,59,m
1,2,54,m
2,3,35,m
3,4,73,f
4,5,49,f


In [9]:
ind['age'] = pd.to_numeric(ind['age'])

ind['age'] = pd.cut(ind['age'], [0,49,120], 
                    labels = ['a0_49','a50+'])
ind

,id,age,sex
0,1,a50+,m
1,2,a50+,m
2,3,a0_49,m
3,4,a50+,f
4,5,a0_49,f


In [10]:
ind.age

0     a50+
1     a50+
2    a0_49
3     a50+
4    a0_49
Name: age, dtype: category
Categories (2, object): ['a0_49' < 'a50+']

In [11]:
con_age.columns

Index(['a0.49', 'a.50+'], dtype='object')

In [12]:
con_age = con_age.rename(columns={'a0.49':'a0_49','a.50+':'a50+'})
con_age

,a0_49,a50+
0,8,4
1,2,8
2,7,4


In [13]:
cons = con_age.merge(con_sex,left_index=True,right_index=True)
cons

,a0_49,a50+,m,f
0,8,4,6,6
1,2,8,4,6
2,7,4,3,8


In [14]:
print ("Shape of ind:",ind.shape)
print ("Shape of cons:",cons.shape)

Shape of ind: (5, 3)
Shape of cons: (3, 4)


In [15]:
age_pivot = pd.pivot_table(ind,columns=['age'],
                           values='id', 
                           index=ind.index, 
                           aggfunc=len, 
                           fill_value=0, 
                           observed=False )
sex_pivot = pd.pivot_table(ind,
                           columns=['sex'],
                           values='id', 
                           index=ind.index, 
                           aggfunc=len, 
                           fill_value=0, 
                           observed=False )[['m','f']]

In [16]:
age_pivot

age,a0_49,a50+
0,0,1
1,0,1
2,1,0
3,0,1
4,1,0


In [49]:
ind_cat = pd.DataFrame(
    age_pivot.to_records()).merge(
    pd.DataFrame(sex_pivot.to_records()),
    left_index=True,right_index=True)

ind_cat = ind_cat.drop(['index_x','index_y'],axis=1)
ind_cat

,a0_49,a50+,m,f
0,0,1,1,0
1,0,1,1,0
2,1,0,1,0
3,0,1,0,1
4,1,0,0,1


In [18]:
ind_cat.sum(axis=0)
ind_agg = ind_cat.sum(axis=0)
ind_agg


a0_49    2
a50+     3
m        3
f        2
dtype: int64

In [19]:
test = pd.concat([cons.iloc[0], ind_agg], 
                 axis=1).transpose()
test

,a0_49,a50+,m,f
0,8,4,6,6
0,2,3,3,2


In [20]:
ind_copy = ind.copy()
ind_copy

,id,age,sex
0,1,a50+,m
1,2,a50+,m
2,3,a0_49,m
3,4,a50+,f
4,5,a0_49,f


In [21]:
ind_copy['weight'] = np.ones(5)
ind_copy

,id,age,sex,weight
0,1,a50+,m,1.0
1,2,a50+,m,1.0
2,3,a0_49,m,1.0
3,4,a50+,f,1.0
4,5,a0_49,f,1.0


In [22]:
ind_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   id      5 non-null      int64   
 1   age     5 non-null      category
 2   sex     5 non-null      object  
 3   weight  5 non-null      float64 
dtypes: category(1), float64(1), int64(1), object(1)
memory usage: 381.0+ bytes


In [26]:
ind_copy['age'] = ind_copy['age'].astype(str)
ind_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      5 non-null      int64  
 1   age     5 non-null      object 
 2   sex     5 non-null      object 
 3   weight  5 non-null      float64
dtypes: float64(1), int64(1), object(2)
memory usage: 292.0+ bytes


In [29]:
age = cons.iloc[0,[0,1]]
sex = cons.iloc[0,[2,3]]

aggregates = [age,sex]
dimensions = [['age'],['sex']]

print(aggregates)
print(dimensions)

[a0_49    8
a50+     4
Name: 0, dtype: int64, m    6
f    6
Name: 0, dtype: int64]
[['age'], ['sex']]


In [50]:
from ipfn import ipfn
ipf = ipfn.ipfn(ind_copy,
                aggregates,
                dimensions,
                weight_col='weight',
                convergence_rate = 1e-15)
out = ipf.iteration()
out

ipfn converged: convergence_rate not updating or below rate_tolerance


,sex,age,id,weight
0,m,a50+,1,1.227998
1,m,a50+,2,1.227998
2,m,a0_49,3,3.544004
3,f,a50+,4,1.544004
4,f,a0_49,5,4.455996


In [34]:
ind_copy_2 = ind.copy()
ind_copy_2['age'] = ind_copy_2['age'].astype(str)

n_zones = len(cons) # number of zones


for i in range(0,n_zones):
    
    ind_copy_2['weight_' + str(i)] = np.ones(5)
    
 
    age = cons.iloc[i,[0,1]]
    sex = cons.iloc[i,[2,3]]
    
    
    ipf = ipfn.ipfn(ind_copy_2, 
                    [age,sex],
                    [['age'],
                     ['sex']],
                    weight_col='weight_'+str(i),
                    convergence_rate = 1e-15)
    ind_copy_2 = ipf.iteration()

ind_copy_2

ipfn converged: convergence_rate not updating or below rate_tolerance
ipfn converged: convergence_rate not updating or below rate_tolerance
ipfn converged: convergence_rate not updating or below rate_tolerance


,sex,age,id,weight_0,weight_1,weight_2
0,m,a50+,1,1.227998,1.725083,0.725083
1,m,a50+,2,1.227998,1.725083,0.725083
2,m,a0_49,3,3.544004,0.549834,1.549834
3,f,a50+,4,1.544004,4.549834,2.549834
4,f,a0_49,5,4.455996,1.450166,5.450166


In [52]:
ind_agg0 = cons.apply(lambda x: 1.0*ind_agg,
                      axis=0).T[0:3].reset_index(drop=True)

ind_agg3 = (ind_agg0 * np.nan).copy()

for i in range(0,n_zones):
    ind_agg3.iloc[i] = ind_cat.apply(lambda x: 
                                     x*ind_copy_2['weight_'+str(i)],
                                     axis=0).sum(axis=0)

ind_agg3

,a0_49,a50+,m,f
0,8.0,4.0,6.0,6.0
1,2.0,8.0,4.0,6.0
2,7.0,4.0,3.0,8.0


In [53]:
cons

,a0_49,a50+,m,f
0,8,4,6,6
1,2,8,4,6
2,7,4,3,8


In [54]:

def int_pp(weights):
    
    xv = np.array(weights).ravel()
    
    rsum = round(xv.sum())
    xs = np.random.choice(len(xv),int(rsum),True,xv/xv.sum())
    
    return np.bincount(xs,minlength=len(xv))



In [39]:
np.random.seed(24)
values = [[0.333, 0.667, 3], [1.333, 1.333, 1.333]]

for i, v in enumerate(values, 1):
    print(f"Results for set {i}:")
    for j in range(5):
        result = int_pp(v)
        print(f"Iteration {j+1}: {result}")
    print()

Results for set 1:
Iteration 1: [0 1 3]
Iteration 2: [0 0 4]
Iteration 3: [0 1 3]
Iteration 4: [0 1 3]
Iteration 5: [0 0 4]

Results for set 2:
Iteration 1: [1 1 2]
Iteration 2: [0 1 3]
Iteration 3: [1 2 1]
Iteration 4: [0 2 2]
Iteration 5: [1 1 2]



In [41]:
def int_trs(weights):
    
    xv = np.array(weights).ravel()
    
    xint = np.floor(xv)
    
    r = xv - xint
    
    frac_sum = round(r.sum())
   
    xs = np.random.choice(len(xv),
                          int(frac_sum),
                          True,
                          r/r.sum())
    
    topup = np.bincount(xs,minlength=len(xv))
    return xint + topup

In [42]:
np.random.seed(24) 
values = [[0.333, 0.667, 3], [1.333, 1.333, 1.333]]

for i, v in enumerate(values, 1):
    print(f"Results for set {i}:")
    for j in range(5):
        result = int_trs(v)
        print(f"Iteration {j+1}: {result}")
    print()

Results for set 1:
Iteration 1: [0. 1. 3.]
Iteration 2: [0. 1. 3.]
Iteration 3: [0. 1. 3.]
Iteration 4: [1. 0. 3.]
Iteration 5: [0. 1. 3.]

Results for set 2:
Iteration 1: [1. 1. 2.]
Iteration 2: [1. 1. 2.]
Iteration 3: [2. 1. 1.]
Iteration 4: [2. 1. 1.]
Iteration 5: [1. 2. 1.]



In [43]:
int_weight1 = int_trs(ind_copy_2['weight_0'])
int_weight1

array([1., 1., 5., 1., 4.])

In [44]:

def int_expand_vector(weights):
    return np.repeat(range(0,len(weights)),
                     weights.astype(int))

print (int_weight1)
print (int_expand_vector(int_weight1))

[1. 1. 5. 1. 4.]
[0 1 2 2 2 2 2 3 4 4 4 4]


In [46]:

exp_indices = int_expand_vector(int_weight1)

ind_orig.iloc[exp_indices]

,id,age,sex,income
0,1,59,m,2868
1,2,54,m,2474
2,3,35,m,2231
2,3,35,m,2231
2,3,35,m,2231
2,3,35,m,2231
2,3,35,m,2231
3,4,73,f,3152
4,5,49,f,2473
4,5,49,f,2473


In [47]:
indivs = []
for i in range(0, n_zones):
    
    ints = int_expand_vector(int_trs(ind_copy_2[
                                     'weight_' + str(i)]))
    
    temp = ind_orig.loc[ints].copy()
    
    temp['zone'] = i
    indivs.append(temp)

ints_df = pd.concat(indivs)

ints_df.reset_index(drop=True, inplace=True)
ints_df

,id,age,sex,income,zone
0,1,59,m,2868,0
1,2,54,m,2474,0
2,3,35,m,2231,0
3,3,35,m,2231,0
4,3,35,m,2231,0
5,4,73,f,3152,0
6,4,73,f,3152,0
7,5,49,f,2473,0
8,5,49,f,2473,0
9,5,49,f,2473,0
